In [ ]:
import os
import csv
import json
import pandas as pd

In [ ]:
df = pd.read_csv('../hierarchical_edge_bundling/tweets_marchandisation.csv', sep=';')

In [ ]:
# Liens de partage et follow

sorted = df.sort_values(by="sha1")
users_parsed = []
edges = "Source;Target;TypeLien;Weight;Sha1\n"
previous_sha1 = None
previous_user = None
not_found = 0

# pour chaque ligne du csv
for index, row in sorted.iterrows():
    user_name = row['from_user_name']
    # si le sha1 de la ligne est celui de la ligne d'avant, on créé un lien
    # cela veut dire que les deux utilisateurs ont partagé la même image
    if previous_sha1 == row['sha1'] and previous_user != user_name:
        edges += f"{user_name};{previous_user};Partage;1;{previous_sha1}\n"
    try:
        # si on a pas déjà regardé les followers de l'utilisateur
        if user_name not in users_parsed:
            # on ouvre le fichier des followers
            with open('following/' + user_name + '.json', 'r') as f:
                # on ne garde que les followers qui sont aussi dans le corpus
                data = [user['username'] for user in json.load(f) if user['username'] in list(df['from_user_name'])]
                # on créé un lien entre chacun de ces followers et l'utilisateur courant
                for user in data:
                    edges += f"{user_name};{user};Follow;0.1;\n"
            users_parsed.append(user_name)
    except FileNotFoundError:
        print(user_name + " not found!")
        not_found += 1
    except json.JSONDecodeError:
        print(user_name + " error!")
    previous_sha1 = row['sha1']
    previous_user = user_name
    not_found += 1

In [14]:
# Liens de partage et follow

sorted = df.sort_values(by="average_hash")
users_parsed = []
edges = "Source;Target;TypeLien;Weight;AverageHash\n"
previous_ahash = None
previous_user = None
not_found = 0

# pour chaque ligne du csv
for index, row in sorted.iterrows():
    user_name = row['from_user_name']
    # si le average hash de la ligne est celui de la ligne d'avant, on créé un lien
    # cela veut dire que les deux utilisateurs ont partagé la même image
    if previous_ahash == row['average_hash'] and previous_user != user_name:
        edges += f"{user_name};{previous_user};Partage;1;{previous_ahash}\n"
    try:
        # si on a pas déjà regardé les followers de l'utilisateur
        if user_name not in users_parsed:
            # on ouvre le fichier des followers
            with open('following/' + user_name + '.json', 'r') as f:
                # on ne garde que les followers qui sont aussi dans le corpus
                data = [user['username'] for user in json.load(
                    f) if user['username'] in list(df['from_user_name'])]
                # on créé un lien entre chacun de ces followers et l'utilisateur courant
                for user in data:
                    edges += f"{user_name};{user};Follow;0.1;\n"
            users_parsed.append(user_name)
    except FileNotFoundError:
        print(user_name + " not found!")
        not_found += 1
    except json.JSONDecodeError:
        print(user_name + " error!")
    previous_ahash = row['average_hash']
    previous_user = user_name
    not_found += 1

In [ ]:
'xarratt' in list(df['from_user_name'])

In [ ]:
# Liens de partage et partage&follow

sorted = df.sort_values(by="sha1")
users_parsed = []
edges = "Source;Target;TypeLien;Weight;Sha1\n"
current_sha1 = None
current_user = None
for index, row in sorted.iterrows():
    user_name = row['from_user_name']
    if current_sha1 == row['sha1'] and current_user != user_name:
        try:
            with open('following/' + user_name + '.json', 'r') as f:
                data = [user['username'] for user in json.load(f)]
                if current_user in data:
                    type_link = "Partage&Follow"
                else:
                    type_link = "Partage"
                edges += f"{user_name};{current_user};{type_link};1;{current_sha1}\n"

        except FileNotFoundError:
            print(user_name + " not found!")
            not_found += 1
        except json.JSONDecodeError:
            print(user_name + " error!")
    current_sha1 = row['sha1']
    current_user = user_name
not_found

In [15]:
with open('edges.csv', 'w', encoding='utf-8') as f:
    f.write(edges)